In [ ]:
# GoogleSheetsAPIの設定
#   GoogleSheetsに書き込むためにGoogleSheetsAPIを使用。以下の手順で設定。
#       1. GoogleCloudConsoleでプロジェクトを作成し，GoogleSheetsAPIを有効にする。
#       2. OAuth2.0クライアントIDを作成し，JSON形式の認証情報をダウンロード。
#       3. ダウンロードしたJSONファイルをプロジェクトディレクトリに保存。

# gspread を利用するには Google サービスへの認証・認可が必要。その方法として以下の２つがある。

# 1. サービスアカウントの利用
# 2. OAuth クライアント ID の利用

# 今回は「サービスアカウント」を利用。
# ※ サービスアカウントとは特殊なタイプのGoogle アカウントのこと。いわゆるBot。
# 　 認証情報はBot(アプリケーション等) に対してGoogle サービスへの認証・認可を提供。
# サービスアカウントは分離された異なるアカウントであるため，デフォルトではスプレッドシートに対するアクセス権限がない。
# 今回はスプレッドシートにアクセスさせたいため，このサービスアカウントと対象のスプレッドシートを共有する。

#  サービスアカウントメアド
# *******


In [1]:
# 必要なライブラリをインポート。
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

# Selenium を使って楽天競馬サイトからデータを取得。
def scrape_rakuten_keiba():
    # ヘッドレスモードで起動するためのオプションを設定。
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    # Chromeを立ち上げる。
    chrome_driver = webdriver.Chrome(options=chrome_options)

    print('ブラウザを起動します。')

    # 今回開くページのurl。
    url = 'https://keiba.rakuten.co.jp/race_card/list/RACEID/202408092135060500'

    # 楽天競馬サイトにアクセス。
    chrome_driver.get(url)

    print('指定のページを開きます。')

    # ページのHTMLをBeautifulSoupで解析。
    soup = BeautifulSoup(chrome_driver.page_source, 'html.parser')

    # ページが完全に読み込まれるまで待機。
    time.sleep(5)
    print('待機中です。')
    
    # 空のリストを作成。
    data = []

    # 例外処理でエラーが出てもブラウザを閉じるようにする。
    try:
        elements = soup.select('#oddsField > table > tbody > tr.race01 > td') 
        race_num = chrome_driver.find_element(By.CSS_SELECTOR, '#oddsField > table > tbody > tr.race01 > th:nth-child(1)').text.strip()
        print(race_num)
        data.append([race_num])

        # サイトからデータを抽出。
        for i in range(2, len(elements) + 2):
            race_date = soup.select_one(f'#oddsField > table > tbody > tr.race01 > td:nth-child({i})').text.strip()
            data.append([race_date])

            print(race_date)
    finally:
        chrome_driver.quit()
        print('ブラウザを終了します。')

    return data

# GoogleSheetsへの認証。
def authenticate_google_sheets():
    scope = [
    'https://www.googleapis.com/auth/spreadsheets', 
    'https://www.googleapis.com/auth/drive'
    ]
    path = '*********.json'
    creds = ServiceAccountCredentials.from_json_keyfile_name(path, scope)
    client = gspread.authorize(creds)
    return client

# スプレッドシートにデータを書き込む。
def write_to_spreadsheet(data):
    client = authenticate_google_sheets()
    sheet = client.open('スクレイピングdemo').sheet1  # スプレッドシートの名前を指定。

    print('スプレッドシートに書き込み中です。')

    for row in data:
        sheet.append_row(row)

def main():
    data = scrape_rakuten_keiba()
    write_to_spreadsheet(data)
    print('書き込みが終了しました。')

main()

# if __name__ == '__main__':
#     main()

ブラウザを起動します。
指定のページを開きます。
待機中です。
1R
15:00
３歳８
ダ1400m
11頭
オッズ
10
10[1]-3[5]-9[4]
過去映像
変更
確定
最大2％還元
ブラウザを終了します。
スプレッドシートに書き込み中です。
書き込みが終了しました。
